# Binder classification with best practices

This notebook revisits the combined SMILES fingerprint + RDKit descriptor workflow and incorporates best practices discussed during the review: data leakage is prevented via pipelines, feature failures are logged for inspection, models account for class imbalance, and nested cross-validation with hyperparameter search is used for unbiased comparison.

In [1]:
import os
os.environ.setdefault('OMP_NUM_THREADS', '1')
os.environ.setdefault('RDKIT_MAX_THREADS', '1')
os.environ.setdefault('RDKIT_DISABLE_THREADS', '1')

import warnings
warnings.filterwarnings('ignore')

import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
from rdkit.ML.Descriptors import MoleculeDescriptors

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import (
    precision_recall_curve,
    roc_curve,
    roc_auc_score,
    auc,
    matthews_corrcoef,
    balanced_accuracy_score,
    accuracy_score,
)
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

plt.rcParams.update({"figure.figsize": (8, 5), "axes.grid": True})


In [2]:
# Load and clean the IC50 dataset

data_path = "ic50.tsv"
df = pd.read_csv(data_path, sep="	", low_memory=False)
df["Standard Value"] = pd.to_numeric(df["Standard Value"], errors="coerce")

df = df.dropna(subset=["Standard Value", "Smiles"]).copy()
df["Binder"] = (df["Standard Value"] <= 2000).astype(int)

unwanted_patterns = [
    ".C", ".Cl", ".NA+", ".Na+", ".na+", "[Na+]", "Cl.", ".O=C(O)C(F)(F)F"
]
mask = pd.Series(False, index=df.index)
for pattern in unwanted_patterns:
    mask |= df["Smiles"].str.contains(pattern, regex=False, na=False)
removed = int(mask.sum())
if removed:
    df = df[~mask].copy()
    print(f"Removed {removed} compounds due to unwanted ion patterns")

dupe_mask = df.duplicated(subset="Smiles", keep="first")
removed_dupes = int(dupe_mask.sum())
if removed_dupes:
    df = df[~dupe_mask].copy()
    print(f"Removed {removed_dupes} duplicate SMILES entries")

df = df.reset_index(drop=True)
print(f"Dataset size after cleaning: {len(df)} compounds")
print(f"Binder prevalence (<=2000 nM): {df['Binder'].mean()*100:.2f}%")
print("Sample molecules:")
for i, (_, row) in enumerate(df.head(5).iterrows(), 1):
    smiles_str = row['Smiles'][:70] + "..." if len(row['Smiles']) > 70 else row['Smiles']
    print(f"  {i}. {smiles_str} | Binder: {row['Binder']}")


Removed 25 compounds due to unwanted ion patterns
Removed 787 duplicate SMILES entries
Dataset size after cleaning: 2039 compounds
Binder prevalence (<=2000 nM): 64.64%
Sample molecules:
  1. CN(C)OC(=O)CCC(=O)O | Binder: 0
  2. CC(=O)CC(=O)CCC(=O)O | Binder: 0
  3. Cc1cccc(C)c1Oc1cc2c(N3CCCC3)nc(-n3cc(C(=O)O)cn3)nc2cc1F | Binder: 1
  4. N#Cc1cccc(NC(=O)c2ccc3cccnc3c2O)c1 | Binder: 0
  5. COc1ccc(CNC(=O)c2ccc3cccnc3c2O)cc1 | Binder: 0


In [3]:
# Morgan fingerprint generation with logging

morgan_generator = GetMorganGenerator(radius=2, fpSize=4096)

fingerprint_rows = []
fingerprint_indices = []
fingerprint_failures = []

print("Converting SMILES to Morgan fingerprints...")
for idx, smiles in df['Smiles'].items():
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        fingerprint_failures.append((idx, smiles))
        continue
    try:
        fp = morgan_generator.GetFingerprint(mol)
        fingerprint_rows.append(np.array(fp, dtype=np.int8))
        fingerprint_indices.append(idx)
    except Exception as exc:
        fingerprint_failures.append((idx, smiles))

X_fp = np.asarray(fingerprint_rows, dtype=np.int8)
y_fp = df.loc[fingerprint_indices, 'Binder'].to_numpy()
print(f"Valid fingerprint count: {X_fp.shape[0]} / {len(df)}")
if fingerprint_failures:
    print(f"Fingerprint failures: {len(fingerprint_failures)} (showing up to 5)")
    for entry in fingerprint_failures[:5]:
        print(f"  idx={entry[0]} smiles={entry[1]}")


Converting SMILES to Morgan fingerprints...
Valid fingerprint count: 2039 / 2039


In [4]:
# RDKit descriptor calculation with deferred imputation and logging

descriptor_names = [name for name, _ in Descriptors._descList]
descriptor_calculator = MoleculeDescriptors.MolecularDescriptorCalculator(descriptor_names)

descriptor_rows = []
descriptor_indices = []
descriptor_failures = []

print("Calculating RDKit descriptors...")
for idx, smiles in df['Smiles'].items():
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        descriptor_failures.append((idx, smiles))
        continue
    try:
        values = descriptor_calculator.CalcDescriptors(mol)
        arr = np.asarray(values, dtype=np.float64)
        arr[~np.isfinite(arr)] = np.nan
        descriptor_rows.append(arr)
        descriptor_indices.append(idx)
    except Exception:
        descriptor_failures.append((idx, smiles))

descriptor_df = pd.DataFrame(descriptor_rows, columns=descriptor_names, index=descriptor_indices)
all_nan_cols = descriptor_df.columns[descriptor_df.isna().all()].tolist()
if all_nan_cols:
    descriptor_df = descriptor_df.drop(columns=all_nan_cols)
    print(f"Dropped {len(all_nan_cols)} all-NaN descriptors")

float32_limit = np.finfo(np.float32).max
large_mask = descriptor_df.abs() >= float32_limit
if large_mask.any().any():
    drop_cols = descriptor_df.columns[large_mask.any(axis=0)].tolist()
    descriptor_df = descriptor_df.drop(columns=drop_cols)
    print(f"Dropped {len(drop_cols)} descriptor(s) exceeding float32 limits")

descriptor_df = descriptor_df.sort_index()
print(f"Descriptor matrix shape: {descriptor_df.shape}")
if descriptor_failures:
    print(f"Descriptor failures: {len(descriptor_failures)} (showing up to 5)")
    for entry in descriptor_failures[:5]:
        print(f"  idx={entry[0]} smiles={entry[1]}")


Calculating RDKit descriptors...
Dropped 1 descriptor(s) exceeding float32 limits
Descriptor matrix shape: (2039, 216)


In [5]:
# Align molecules with both feature sets

fp_idx_map = {idx: pos for pos, idx in enumerate(fingerprint_indices)}
desc_idx_map = {idx: pos for pos, idx in enumerate(descriptor_df.index.tolist())}
shared_indices = sorted(set(fp_idx_map) & set(desc_idx_map))
print(f"Molecules with both fingerprints and descriptors: {len(shared_indices)}")
if len(shared_indices) < len(df):
    missing = len(df) - len(shared_indices)
    print(f"Excluded {missing} compounds lacking at least one feature representation")

X_fp_shared = np.asarray([X_fp[fp_idx_map[idx]] for idx in shared_indices], dtype=np.float32)
X_desc_shared = descriptor_df.loc[shared_indices].to_numpy(dtype=np.float32)
y_shared = df.loc[shared_indices, 'Binder'].to_numpy()

fp_dim = X_fp_shared.shape[1]
desc_dim = X_desc_shared.shape[1]
X_combined = np.hstack([X_fp_shared, X_desc_shared]).astype(np.float32)
print(f"Combined feature matrix: {X_combined.shape} (fp_dim={fp_dim}, desc_dim={desc_dim})")


Molecules with both fingerprints and descriptors: 2039
Combined feature matrix: (2039, 4312) (fp_dim=4096, desc_dim=216)


In [6]:
# Preprocessing transformer and evaluation helpers

class CombinedFeaturePreprocessor(BaseEstimator, TransformerMixin):
    "Impute + scale descriptor block while leaving fingerprint bits untouched."

    def __init__(self, fp_dim, epsilon=1e-12):
        self.fp_dim = fp_dim
        self.epsilon = epsilon

    def fit(self, X, y=None):
        X = np.asarray(X)
        desc = X[:, self.fp_dim:]
        medians = np.nanmedian(desc, axis=0)
        medians = np.where(np.isnan(medians), 0.0, medians)
        desc_imputed = np.where(np.isnan(desc), medians, desc)
        mean = np.nanmean(desc_imputed, axis=0)
        mean = np.where(np.isnan(mean), 0.0, mean)
        scale = np.nanstd(desc_imputed, axis=0)
        scale = np.where((scale < self.epsilon) | np.isnan(scale), 1.0, scale)
        self.medians_ = medians
        self.mean_ = mean
        self.scale_ = scale
        return self

    def transform(self, X):
        X = np.asarray(X)
        fp_block = X[:, :self.fp_dim]
        desc = X[:, self.fp_dim:]
        desc_imputed = np.where(np.isnan(desc), self.medians_, desc)
        desc_scaled = (desc_imputed - self.mean_) / self.scale_
        desc_scaled = np.nan_to_num(desc_scaled, copy=False)
        return np.hstack([fp_block, desc_scaled]).astype(np.float32)

def compute_metrics(y_true, y_prob, threshold=0.5):
    y_pred = (y_prob >= threshold).astype(int)
    precision, recall, _ = precision_recall_curve(y_true, y_prob)
    return {
        "PR_AUC": auc(recall, precision),
        "ROC_AUC": roc_auc_score(y_true, y_prob),
        "MCC": matthews_corrcoef(y_true, y_pred),
        "BalancedAccuracy": balanced_accuracy_score(y_true, y_pred),
        "Accuracy": accuracy_score(y_true, y_pred),
    }

preprocessor = CombinedFeaturePreprocessor(fp_dim=fp_dim)
print(preprocessor)


CombinedFeaturePreprocessor(fp_dim=4096)


In [7]:
# Model registry with class-weighted estimators and hyperparameter spaces

MAX_SEARCH_ITER = 12  # cap per-model RandomizedSearch iterations to keep runtime manageable

def build_pipeline(estimator):
    return Pipeline([
        ("preprocess", CombinedFeaturePreprocessor(fp_dim=fp_dim)),
        ("clf", estimator),
    ])

model_spaces = {
    "RandomForest": {
        "pipeline": build_pipeline(RandomForestClassifier(
            n_estimators=500,
            class_weight='balanced',
            random_state=42,
            n_jobs=-1,
        )),
        "param_distributions": {
            "clf__n_estimators": [300, 400, 500, 600, 700, 800, 900],
            "clf__max_depth": [None, 12, 16, 20, 24, 28],
            "clf__min_samples_leaf": [1, 2, 3, 4],
            "clf__min_samples_split": [2, 4, 6, 8],
            "clf__max_features": ['sqrt', 'log2', 0.3, 0.5],
        },
        "n_iter": 20,
    },
    "ExtraTrees": {
        "pipeline": build_pipeline(ExtraTreesClassifier(
            n_estimators=600,
            class_weight='balanced',
            random_state=42,
            n_jobs=-1,
        )),
        "param_distributions": {
            "clf__n_estimators": [400, 600, 800, 1000],
            "clf__max_depth": [None, 12, 16, 20, 28],
            "clf__min_samples_leaf": [1, 2, 3, 5],
            "clf__max_features": ['sqrt', 'log2', 0.2, 0.4],
            "clf__bootstrap": [True, False],
        },
        "n_iter": 20,
    },
    "GradientBoosting": {
        "pipeline": build_pipeline(GradientBoostingClassifier(
            random_state=42,
            subsample=0.8,
        )),
        "param_distributions": {
            "clf__learning_rate": np.linspace(0.01, 0.2, 10),
            "clf__n_estimators": [200, 300, 400, 500],
            "clf__max_depth": [2, 3, 4, 5],
            "clf__min_samples_leaf": [1, 2, 3],
            "clf__max_features": ['sqrt', 'log2', 0.3, 0.5],
        },
        "n_iter": 15,
    },
    "LogisticRegression": {
        "pipeline": build_pipeline(LogisticRegression(
            penalty='elasticnet',
            solver='saga',
            class_weight='balanced',
            max_iter=5000,
            l1_ratio=0.5,
            random_state=42,
        )),
        "param_distributions": {
            "clf__C": np.logspace(-3, 1, 20),
            "clf__l1_ratio": np.linspace(0.1, 0.9, 9),
        },
        "n_iter": 20,
    },
}

model_spaces


{'RandomForest': {'pipeline': Pipeline(steps=[('preprocess', CombinedFeaturePreprocessor(fp_dim=4096)),
                  ('clf',
                   RandomForestClassifier(class_weight='balanced',
                                          n_estimators=500, n_jobs=-1,
                                          random_state=42))]),
  'param_distributions': {'clf__n_estimators': [300,
    400,
    500,
    600,
    700,
    800,
    900],
   'clf__max_depth': [None, 12, 16, 20, 24, 28],
   'clf__min_samples_leaf': [1, 2, 3, 4],
   'clf__min_samples_split': [2, 4, 6, 8],
   'clf__max_features': ['sqrt', 'log2', 0.3, 0.5]},
  'n_iter': 20},
 'ExtraTrees': {'pipeline': Pipeline(steps=[('preprocess', CombinedFeaturePreprocessor(fp_dim=4096)),
                  ('clf',
                   ExtraTreesClassifier(class_weight='balanced', n_estimators=600,
                                        n_jobs=-1, random_state=42))]),
  'param_distributions': {'clf__n_estimators': [400, 600, 800, 1000],
   '

In [8]:
# Nested cross-validation with hyperparameter search

outer_splits = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=42).split(X_combined, y_shared))
results_frames = []

for model_name, spec in model_spaces.items():
    print(f"=== {model_name} ===")
    fold_rows = []
    for fold_id, (train_idx, test_idx) in enumerate(outer_splits, 1):
        inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=fold_id * 31)
        n_iter = min(spec.get("n_iter", 15), MAX_SEARCH_ITER)
        search = RandomizedSearchCV(
            estimator=spec["pipeline"],
            param_distributions=spec["param_distributions"],
            n_iter=n_iter,
            scoring='roc_auc',
            cv=inner_cv,
            n_jobs=-1,
            refit=True,
            random_state=fold_id * 17,
            verbose=0,
        )
        start = time.perf_counter()
        search.fit(X_combined[train_idx], y_shared[train_idx])
        train_seconds = time.perf_counter() - start
        best_model = search.best_estimator_
        y_prob = best_model.predict_proba(X_combined[test_idx])[:, 1]
        metrics = compute_metrics(y_shared[test_idx], y_prob)
        fold_rows.append({
            "Model": model_name,
            "Fold": fold_id,
            **metrics,
            "TrainSeconds": train_seconds,
            "BestParams": search.best_params_,
        })
        print(
            f"Fold {fold_id}: ROC-AUC={metrics['ROC_AUC']:.3f}, PR-AUC={metrics['PR_AUC']:.3f}, "
            f"Train={train_seconds:.1f}s (n_iter={n_iter})"
        )
    fold_df = pd.DataFrame(fold_rows)
    results_frames.append(fold_df)

detailed_results = pd.concat(results_frames, ignore_index=True)
print("Detailed nested CV results:")
display(detailed_results)


=== RandomForest ===
Fold 1: ROC-AUC=0.954, PR-AUC=0.970, Train=186.6s (n_iter=12)
Fold 2: ROC-AUC=0.987, PR-AUC=0.993, Train=238.1s (n_iter=12)
Fold 3: ROC-AUC=0.977, PR-AUC=0.988, Train=275.1s (n_iter=12)
Fold 4: ROC-AUC=0.984, PR-AUC=0.992, Train=213.9s (n_iter=12)
Fold 5: ROC-AUC=0.980, PR-AUC=0.990, Train=288.4s (n_iter=12)
=== ExtraTrees ===
Fold 1: ROC-AUC=0.960, PR-AUC=0.974, Train=335.6s (n_iter=12)
Fold 2: ROC-AUC=0.985, PR-AUC=0.992, Train=299.5s (n_iter=12)
Fold 3: ROC-AUC=0.976, PR-AUC=0.989, Train=252.0s (n_iter=12)
Fold 4: ROC-AUC=0.986, PR-AUC=0.993, Train=290.3s (n_iter=12)
Fold 5: ROC-AUC=0.982, PR-AUC=0.991, Train=183.0s (n_iter=12)
=== GradientBoosting ===
Fold 1: ROC-AUC=0.954, PR-AUC=0.970, Train=230.7s (n_iter=12)
Fold 2: ROC-AUC=0.986, PR-AUC=0.992, Train=184.9s (n_iter=12)
Fold 3: ROC-AUC=0.980, PR-AUC=0.990, Train=39.7s (n_iter=12)
Fold 4: ROC-AUC=0.982, PR-AUC=0.991, Train=60.6s (n_iter=12)
Fold 5: ROC-AUC=0.976, PR-AUC=0.988, Train=132.5s (n_iter=12)
=== Log

,Model,Fold,PR_AUC,ROC_AUC,MCC,BalancedAccuracy,Accuracy,TrainSeconds,BestParams
0,RandomForest,1,0.970263,0.954110,0.769002,0.882667,0.894608,186.601218,"{'clf__n_estimators': 800, 'clf__min_samples_s..."
1,RandomForest,2,0.992577,0.986585,0.859794,0.927083,0.936275,238.089143,"{'clf__n_estimators': 700, 'clf__min_samples_s..."
2,RandomForest,3,0.988295,0.976668,0.840216,0.922664,0.926471,275.121467,"{'clf__n_estimators': 700, 'clf__min_samples_s..."
3,RandomForest,4,0.991990,0.983546,0.851590,0.929609,0.931373,213.871240,"{'clf__n_estimators': 800, 'clf__min_samples_s..."
4,RandomForest,5,0.990027,0.980315,0.845688,0.926014,0.928747,288.429672,"{'clf__n_estimators': 600, 'clf__min_samples_s..."
5,ExtraTrees,1,0.974353,0.959932,0.789655,0.890271,0.904412,335.579293,"{'clf__n_estimators': 800, 'clf__min_samples_l..."
6,ExtraTrees,2,0.991520,0.985085,0.849321,0.923295,0.931373,299.464597,"{'clf__n_estimators': 1000, 'clf__min_samples_..."
7,ExtraTrees,3,0.988657,0.976326,0.849497,0.932449,0.928922,252.048731,"{'clf__n_estimators': 800, 'clf__min_samples_l..."
8,ExtraTrees,4,0.993079,0.986045,0.872803,0.946338,0.938725,290.329387,"{'clf__n_estimators': 1000, 'clf__min_samples_..."
9,ExtraTrees,5,0.990583,0.982375,0.867728,0.938332,0.938575,182.985635,"{'clf__n_estimators': 400, 'clf__min_samples_l..."


In [9]:
# Aggregate metrics (mean +/- std) per model

metric_cols = ["PR_AUC", "ROC_AUC", "MCC", "BalancedAccuracy", "Accuracy", "TrainSeconds"]
summary = (detailed_results
           .groupby("Model")[metric_cols]
           .agg(['mean', 'std'])
           .sort_values(("ROC_AUC", "mean"), ascending=False))
summary.columns = [f"{metric}_{stat}" for metric, stat in summary.columns]
summary = summary.reset_index()
print("Model performance summary (nested CV):")
display(summary)


Model performance summary (nested CV):


,Model,PR_AUC_mean,PR_AUC_std,ROC_AUC_mean,ROC_AUC_std,MCC_mean,MCC_std,BalancedAccuracy_mean,BalancedAccuracy_std,Accuracy_mean,Accuracy_std,TrainSeconds_mean,TrainSeconds_std
0,ExtraTrees,0.987638,0.007597,0.977953,0.010762,0.845801,0.033122,0.926137,0.021743,0.928401,0.014095,272.081528,57.996065
1,RandomForest,0.986630,0.009305,0.976245,0.012912,0.833258,0.036647,0.917607,0.019691,0.923494,0.016555,240.422548,42.177753
2,GradientBoosting,0.986324,0.009097,0.975616,0.012337,0.828976,0.034618,0.912389,0.019430,0.922020,0.015542,129.684245,80.837955
3,LogisticRegression,0.980520,0.011455,0.971350,0.011152,0.821509,0.046988,0.915761,0.025554,0.916621,0.021464,818.847507,252.919964


In [10]:
# Presentation-style performance table

pretty_table = []
metric_pairs = [
    ("ROC_AUC", "Area Under ROC"),
    ("PR_AUC", "Area Under PR"),
    ("BalancedAccuracy", "Balanced Accuracy"),
    ("Accuracy", "Accuracy"),
    ("MCC", "Matthews CC"),
]
for model, group in detailed_results.groupby("Model"):
    row = {"Model": model}
    for metric, label in metric_pairs:
        mean_val = group[metric].mean()
        std_val = group[metric].std()
        row[f"{label}"] = f"{mean_val:.3f} ± {std_val:.3f}"
    mean_time = group["TrainSeconds"].mean()
    std_time = group["TrainSeconds"].std()
    row["Training Time (s)"] = f"{mean_time:.1f} ± {std_time:.1f}"
    best_idx = group["ROC_AUC"].idxmax()
    row["Representative Params"] = detailed_results.loc[best_idx, "BestParams"]
    pretty_table.append(row)

presentation_df = pd.DataFrame(pretty_table).sort_values("Area Under ROC", ascending=False).reset_index(drop=True)
print("Professional summary table (mean ± std across outer folds):")
display(presentation_df)


Professional summary table (mean ± std across outer folds):


,Model,Area Under ROC,Area Under PR,Balanced Accuracy,Accuracy,Matthews CC,Training Time (s),Representative Params
0,ExtraTrees,0.978 ± 0.011,0.988 ± 0.008,0.926 ± 0.022,0.928 ± 0.014,0.846 ± 0.033,272.1 ± 58.0,"{'clf__n_estimators': 1000, 'clf__min_samples_..."
1,RandomForest,0.976 ± 0.013,0.987 ± 0.009,0.918 ± 0.020,0.923 ± 0.017,0.833 ± 0.037,240.4 ± 42.2,"{'clf__n_estimators': 700, 'clf__min_samples_s..."
2,GradientBoosting,0.976 ± 0.012,0.986 ± 0.009,0.912 ± 0.019,0.922 ± 0.016,0.829 ± 0.035,129.7 ± 80.8,"{'clf__n_estimators': 200, 'clf__min_samples_l..."
3,LogisticRegression,0.971 ± 0.011,0.981 ± 0.011,0.916 ± 0.026,0.917 ± 0.021,0.822 ± 0.047,818.8 ± 252.9,"{'clf__l1_ratio': 0.8, 'clf__C': 3.79269019073..."
